In [4]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Hàm tải dữ liệu từ thư mục ảnh
def load_data(data_dir):
    images = []
    labels = []
    for filename in os.listdir(data_dir):
        if filename.endswith(".jpg"):
            label = filename.split("_")[0]
            if label in ["right", "forward", "left"]:
                img = cv2.imread(os.path.join(data_dir, filename))
                img = cv2.resize(img, (128, 128))  # Resize về kích thước 128x128
                images.append(img)
                labels.append(["right", "forward", "left"].index(label))
    return np.array(images), np.array(labels)

# Tải và chia dữ liệu
data_dir = "dataset"  # Thay bằng đường dẫn thực tế
images, labels = load_data(data_dir)
train_images, val_images, train_labels, val_labels = train_test_split(
    images, labels, test_size=0.2, random_state=42
)

# Chuẩn hóa dữ liệu (giá trị pixel từ [0,255] về [0,1])
train_images = train_images / 255.0
val_images = val_images / 255.0

# Tăng cường dữ liệu để chống overfitting
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)
train_generator = train_datagen.flow(train_images, train_labels, batch_size=32)

# Xây dựng mô hình CNN
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation="relu", input_shape=(128, 128, 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(64, (3, 3), activation="relu"),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(128, (3, 3), activation="relu"),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    
    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.5),  # Dropout để chống overfitting
    layers.Dense(3, activation="softmax")  # 3 lớp: right, forward, left
])

# Compile mô hình
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# Huấn luyện mô hình
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=(val_images, val_labels),
    steps_per_epoch=len(train_images) // 32
)

# Vẽ đồ thị loss
plt.plot(history.history["loss"], label="train_loss")
plt.plot(history.history["val_loss"], label="val_loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training and Validation Loss")
plt.legend()
plt.savefig("cnn_loss_plot.png")
plt.close()

# Lưu mô hình dưới dạng .tflite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open("cnn_model.tflite", "wb") as f:
    f.write(tflite_model)


c:\Users\nguye\anaconda3\envs\dip_env\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\nguye\anaconda3\envs\dip_env\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 12s 701ms/step - accuracy: 0.3870 - loss: 6.1026 - val_accuracy: 0.1182 - val_loss: 1.3441
Epoch 2/50
 1/13 ━━━━━━━━━━━━━━━━━━━━ 8s 676ms/step - accuracy: 0.3750 - loss: 2.9916

c:\Users\nguye\anaconda3\envs\dip_env\lib\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - accuracy: 0.3750 - loss: 2.9916 - val_accuracy: 0.1818 - val_loss: 1.2736
Epoch 3/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 9s 646ms/step - accuracy: 0.4503 - loss: 2.5837 - val_accuracy: 0.2727 - val_loss: 3.5649
Epoch 4/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - accuracy: 0.2812 - loss: 1.6069 - val_accuracy: 0.2727 - val_loss: 4.0664
Epoch 5/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 8s 642ms/step - accuracy: 0.4516 - loss: 1.4329 - val_accuracy: 0.2727 - val_loss: 9.2493
Epoch 6/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.4688 - loss: 1.3118 - val_accuracy: 0.2727 - val_loss: 9.6637
Epoch 7/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 8s 632ms/step - accuracy: 0.5014 - loss: 1.2587 - val_accuracy: 0.2727 - val_loss: 14.7513
Epoch 8/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.5312 - loss: 1.3151 - val_accuracy: 0.2727 - val_loss: 14.8322
Epoch 9/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 9s 664ms/step - accuracy: 0.5711 - loss: 1.1473 - val_accuracy: 0.2727 - val_lo

INFO:tensorflow:Assets written to: C:\Users\nguye\AppData\Local\Temp\tmpgu9cch_a\assets


Saved artifact at 'C:\Users\nguye\AppData\Local\Temp\tmpgu9cch_a'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name='keras_tensor_172')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  2019313534224: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2019291713200: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2019291717776: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2019291719536: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2019291716896: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2019291718304: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2020614039056: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2019291720416: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2019291722176: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2019291721648: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2

In [ ]:

# Tải mô hình .tflite
interpreter = tf.lite.Interpreter(model_path="cnn_model.tflite")
interpreter.allocate_tensors()

# Lấy thông tin input và output
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Kiểm tra trên luồng video
stream_url = "http://192.168.46.130:81/stream"
cap = cv2.VideoCapture(stream_url)

while True:
    ret, frame = cap.read()
    if not ret:
        print("Không thể đọc luồng video.")
        break

    # Chuẩn bị frame
    frame_resized = cv2.resize(frame, (128, 128))
    frame_processed = frame_resized / 255.0
    input_data = np.expand_dims(frame_processed, axis=0).astype(np.float32)

    # Chạy dự đoán
    interpreter.set_tensor(input_details[0]["index"], input_data)
    interpreter.invoke()

    # Lấy kết quả
    output_data = interpreter.get_tensor(output_details[0]["index"])
    prediction = np.argmax(output_data[0])
    result = ["right", "forward", "left"][prediction]
    print(f"Dự đoán: {result}")

    # Hiển thị frame (tùy chọn)
    cv2.imshow("Video Stream", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):  # Nhấn 'q' để thoát
        break

# Giải phóng tài nguyên
cap.release()
cv2.destroyAllWindows()